 1. Library yang digunakan

In [23]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

2. Load Dataset

In [2]:
!pip install xlrd


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
stunting_dataset = pd.read_csv('databalita.csv')

In [4]:
stunting_dataset.head()

Umur (bulan) Jenis Kelamin  Tinggi Badan (cm)       Status Gizi
0             0     laki-laki          44.591973           stunted
1             0     laki-laki          56.705203            tinggi
2             0     laki-laki          46.863358            normal
3             0     laki-laki          47.508026            normal
4             0     laki-laki          42.743494  severely stunted

In [5]:
category_counts = stunting_dataset['Status Gizi'].value_counts()
category_counts

Status Gizi
normal              67755
severely stunted    19869
tinggi              19560
stunted             13815
Name: count, dtype: int64

In [7]:
# mengkodekan jenis kelamin
stunting_dataset['Jenis Kelamin'] = stunting_dataset['Jenis Kelamin'].map({'laki-laki': 0, 'perempuan': 1})

# mengkodekan status gizi
stunting_dataset['Status Gizi'] = stunting_dataset['Status Gizi'].map({'severely stunted': 0, 'stunted': 1, 'normal': 2, 'tinggi': 3})
stunting_dataset

Umur (bulan)  Jenis Kelamin  Tinggi Badan (cm)  Status Gizi
0                  0              0          44.591973            1
1                  0              0          56.705203            3
2                  0              0          46.863358            2
3                  0              0          47.508026            2
4                  0              0          42.743494            0
...              ...            ...                ...          ...
120994            60              1         100.600000            2
120995            60              1          98.300000            1
120996            60              1         121.300000            2
120997            60              1         112.200000            2
120998            60              1         109.800000            2

[120999 rows x 4 columns]

In [9]:
# memisahkan fitur dan target
X = stunting_dataset.iloc[:, 1:6]
X = stunting_dataset.drop('Status Gizi', axis=1)
y = stunting_dataset['Status Gizi']

# membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

Umur (bulan)  Jenis Kelamin  Tinggi Badan (cm)
0                  0              0          44.591973
1                  0              0          56.705203
2                  0              0          46.863358
3                  0              0          47.508026
4                  0              0          42.743494
...              ...            ...                ...
120994            60              1         100.600000
120995            60              1          98.300000
120996            60              1         121.300000
120997            60              1         112.200000
120998            60              1         109.800000

[120999 rows x 3 columns]

3. Membuat Model Menggunakan Algoritma Random Forest

In [10]:
classifier = RandomForestClassifier(n_estimators=100)

In [11]:
# melatih model Random Forest dengan data latih
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
# membuat prediksi dengan data uji
y_pred = classifier.predict(X_test)

# menghitung akurasi model
accuracy = accuracy_score(y_test, y_pred)
print('Akurasi model Random Forest:', accuracy)

Akurasi model Random Forest: 0.9991735537190083


In [15]:
print('confusion_matrix Random Forest:')
confusion_matrix(y_test, y_pred)

confusion_matrix Random Forest:


array([[ 4130,     0,     0,     0],
       [    4,  2785,     1,     0],
       [    0,     5, 13376,     1],
       [    0,     0,     9,  3889]])

In [16]:
print(classification_report(y_test.values.reshape(-1, 1), y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4130
           1       1.00      1.00      1.00      2790
           2       1.00      1.00      1.00     13382
           3       1.00      1.00      1.00      3898

    accuracy                           1.00     24200
   macro avg       1.00      1.00      1.00     24200
weighted avg       1.00      1.00      1.00     24200



In [18]:
print('Cross val: ', cross_val_score(classifier, y_test.values.reshape(-1, 1), y_pred, cv = 5))
print('Accuracy : ', np.mean(cross_val_score(classifier, y_test.values.reshape(-1, 1), y_pred)))

Cross val:  [0.99896694 0.99917355 0.99938017 0.99938017 0.99896694]
Accuracy :  0.9991735537190083


4. Membuat Model Prediksi

In [20]:
# Testing model
test = pd.DataFrame({'Umur (bulan)': [20], 'Jenis Kelamin': [1], 'Tinggi Badan (cm)': [86.2]})

# Melakukan prediksi
pred = classifier.predict(test)
print("Predicted Score = {}".format(np.array(pred)))

# Menginterpretasikan hasil prediksi
if pred[0] == 0:
    print('Status Gizi : Severely Stunted')
elif pred[0] == 1:
    print('Status Gizi : Stunted')
elif pred[0] == 2:
    print('Status Gizi : Normal')
else:
    print('Status Gizi : Tinggi')

Predicted Score = [2]
Status Gizi : Normal


5. Simpan Model

In [24]:
filename = 'stunting_risk_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [25]:
filename = 'scaler.pkl'
pickle.dump(classifier, open(filename, 'wb'))
